# Ranking and Position Sizing

**PyBroker** includes support for ranking ticker symbols and setting position sizes on a group of symbols. This notebook goes over how to use both of these features in your trading strategy.

In [1]:
import pybroker
from pybroker import Strategy, StrategyConfig, YFinance

pybroker.enable_data_source_cache('ranking_and_pos_sizing')

## Ranking Ticker Symbols

First, we will look at ranking ticker symbols when placing a buy order.

In [2]:
def buy_highest_volume(ctx):
    # If there are no long positions across all tickers being traded:
    if not tuple(ctx.long_positions()):
        ctx.buy_shares = ctx.calc_target_shares(1)
        ctx.hold_bars = 2
        ctx.score = ctx.volume[-1]

The above ranks ticker symbols by their most recent trading volume, allocating 100% of the portfolio for 2 days.

In [3]:
config = StrategyConfig(max_long_positions=1)
strategy = Strategy(YFinance(), '6/1/2021', '6/1/2022', config)
strategy.add_execution(buy_highest_volume, ['T', 'F', 'GM', 'PFE'])

When creating the [Strategy](https://pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy), we set [max_long_positions](https://pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.max_long_positions) to ```1```. This limits the number of long positions that can be held at any time to 1. The result is only placing a buy order for the ticker that has the highest volume from ```['T', 'F', 'GM', 'PFE']```:

In [4]:
result = strategy.backtest(calc_bootstrap=False)
result.trades

Backtesting: 2021-06-01 00:00:00 to 2022-06-01 00:00:00

Loading bar data...
[*********************100%***********************]  4 of 4 completed
Loaded bar data: 0:00:02 

Test split: 2021-06-01 04:00:00 to 2022-05-31 04:00:00


100% (253 of 253) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:05


,type,symbol,entry_date,exit_date,shares,pnl,return_pct,cumulative_pnl,bars,pnl_per_bar
id,,,,,,,,,,
1,long,F,2021-06-02 04:00:00,2021-06-04 04:00:00,6734,8619.52,8.62,8619.52,2,4309.76
2,long,F,2021-06-07 04:00:00,2021-06-09 04:00:00,6801,-2856.42,-2.64,5763.10,2,-1428.21
3,long,F,2021-06-10 04:00:00,2021-06-14 04:00:00,6832,-2527.84,-2.40,3235.26,2,-1263.92
4,long,F,2021-06-15 04:00:00,2021-06-17 04:00:00,6900,207.00,0.20,3442.26,2,103.50
5,long,F,2021-06-18 04:00:00,2021-06-22 04:00:00,7003,2451.05,2.40,5893.31,2,1225.53
...,...,...,...,...,...,...,...,...,...,...
80,long,F,2022-05-10 04:00:00,2022-05-12 04:00:00,7263,-6972.48,-7.15,-9423.13,2,-3486.24
81,long,F,2022-05-13 04:00:00,2022-05-17 04:00:00,6835,615.15,0.68,-8807.98,2,307.58
82,long,F,2022-05-18 04:00:00,2022-05-20 04:00:00,6739,-2965.16,-3.38,-11773.14,2,-1482.58


## Setting Position Sizes

It is also possible to set position sizes based on multiple tickers. Let's start with a simple buy and hold strategy that begins trading after 100 days, and holds the position for 30 days.

In [5]:
def buy_and_hold(ctx):
    if not ctx.long_pos() and len(ctx.close) > 100:
        ctx.buy_shares = 100
        ctx.hold_bars = 30
        
strategy = Strategy(YFinance(), '6/1/2021', '6/1/2022')
strategy.add_execution(buy_and_hold, ['T', 'F', 'GM', 'PFE'])

The above will buy ```100``` shares in each of ```['T', 'F', 'GM', 'PFE']```. But what if we do not want to use equal position sizing? For example, we may want to size positions so that more shares are allocated to tickers with lower volatility to try to decrease the portfolio's overall volatility.

We define a function that does just that below:

In [6]:
import numpy as np

def pos_size_handler(ctx):
    # Fetch all buy signals.
    signals = tuple(ctx.signals("buy"))
    # Return if there are no buy signals (i.e. there are only sell signals).
    if not signals:
        return
    # Calculates the inverse volatility, where volatility is defined as the
    # standard deviation of close prices for the last 100 days.
    get_inverse_volatility = lambda signal: 1 / np.std(signal.bar_data.close[-100:])
    # Sums the inverse volatilities for all of the buy signals.
    total_inverse_volatility = sum(map(get_inverse_volatility, signals))
    for signal in signals:
        size = get_inverse_volatility(signal) / total_inverse_volatility
        # Calculate the number of shares given the latest close price.
        shares = ctx.calc_target_shares(size, signal.bar_data.close[-1])
        ctx.set_shares(signal, shares)
        
strategy.set_pos_size_handler(pos_size_handler)

The [pos_size_handler](https://pybroker.com/en/latest/reference/pybroker.strategy.html#pybroker.strategy.Strategy.set_pos_size_handler) will run on every bar that generates a buy or sell signal when [buy_shares](https://pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.buy_shares) or [sell_shares](https://pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.sell_shares) is set on the [ExecContext](https://pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext).

In [7]:
result = strategy.backtest(calc_bootstrap=False)

Backtesting: 2021-06-01 00:00:00 to 2022-06-01 00:00:00

Loaded cached bar data.

Test split: 2021-06-01 04:00:00 to 2022-05-31 04:00:00


100% (253 of 253) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


In [8]:
result.trades

,type,symbol,entry_date,exit_date,shares,pnl,return_pct,cumulative_pnl,bars,pnl_per_bar
id,,,,,,,,,,
1,long,GM,2021-10-22 04:00:00,2021-12-06 05:00:00,140,217.00,2.66,217.00,30,7.23
2,long,T,2021-10-22 04:00:00,2021-12-06 05:00:00,2391,-3706.05,-8.01,-3489.05,30,-123.54
3,long,PFE,2021-10-22 04:00:00,2021-12-06 05:00:00,251,2424.66,22.51,-1064.39,30,80.82
4,long,F,2021-10-22 04:00:00,2021-12-06 05:00:00,2123,5647.18,16.23,4582.79,30,188.24
5,long,GM,2021-12-07 05:00:00,2022-01-20 05:00:00,224,-1267.84,-9.20,3314.95,30,-42.26
6,long,T,2021-12-07 05:00:00,2022-01-20 05:00:00,2947,8929.41,17.31,12244.36,30,297.65
7,long,PFE,2021-12-07 05:00:00,2022-01-20 05:00:00,321,754.35,4.57,12998.71,30,25.15
8,long,F,2021-12-07 05:00:00,2022-01-20 05:00:00,1140,2770.20,12.28,15768.91,30,92.34
9,long,GM,2022-01-21 05:00:00,2022-03-07 05:00:00,321,-4044.60,-23.47,11724.31,30,-134.82


Setting position sizes in this manner opens up a lot of possibilities, such as using [Mean-Variance Optimization](https://en.wikipedia.org/wiki/Modern_portfolio_theory) to determine portfolio allocations!

[In the next notebook, we will switch gears and discuss how to implement custom indicators in **PyBroker**.](https://pybroker.readthedocs.io/en/latest/notebooks/5.%20Writing%20Indicators.html)